<h1 style="text-align:center; font-weight:bold; color:#2E86C1;">Project 2</h1>


## Task 1



### 2nd Order ODE problem
$M \ddot{u}_{n+1} =  f_{n+1} - C \dot{u}_{n+1} - K u_{n+1}$

### Brief Overview Newmark-β

The Newmark-β method is a numerical method used to solve second order ODEs with taylor expansion in combination with time integration. Approximating the time integration with the generelized trapezoidal rule leads to the update relations:

\begin{equation}
\dot{u}_{n+1} - \dot{u}_n = \int_{t_n}^{t_{n+1}} \ddot{u}(t) \, dt \approx (1 - \gamma) h \ddot{u}_n + \gamma h \ddot{u}_{n+1}
\end{equation}

\begin{equation}
u_{n+1} - u_n = h \dot{u}_n + h^2 \left[ (1 - 2\beta) \ddot{u}_n + 2\beta \ddot{u}_{n+1} \right] / 2
\end{equation}

We now have three equations and three unknowns: $u_{n+1}, \dot{u}_{n+1}, \ddot{u}_{n+1}$. Depending on the chosen parameters $\gamma$, $\beta$, and (for HHT‑α) $\alpha$, the resulting scheme exhibits different stability and accuracy properties. Note that if $\beta = 1/2$ the generelized trapezoidal rule causes the scheme to be implicit and if $\beta = 0$ the method is explicit.

### Brief Overview HHT-α
The Newmark-β method can be further generelized by modifiying the equations of motion by introducing the dissipation parameter $\alpha \in [-1/3, 0]$ which dampens unwanted high-frequence modes by numerical dampening while still achiving second-order accuracy.


### Case Distinctions

#### Case A: Implicit Method with Damping $C \neq 0 $
If $C \neq 0 $ and we use the implicit Newmark-β version, i.e $\beta > 1/4$ we solve the system of equations by expressing $\ddot{u}_{n+1}$ and $\dot{u}_{n+1}$ in terms of $u_{n+1}$ and inserting these into $M \ddot{u}_{n+1} =  f_{n+1} - C \dot{u}_{n+1} - K u_{n+1}$. This yields the effective system: 

\begin{equation} 
\underbrace{\left[ \frac{M}{\beta \Delta t^2} + \frac{\gamma C}{\beta \Delta t} + K \right]}_{\text{effective stiffness matrix}} u_{n+1} + M \left[ \frac{u_n}{\beta \Delta t^2} + \frac{\dot{u}_n}{\beta \Delta t} + \left( \frac{1}{2\beta} - 1 \right) \ddot{u}_n \right] + C \left[ \frac{\gamma u_n}{\beta \Delta t} - \left( 1 - \frac{\gamma}{\beta} \right) \dot{u}_n - \left( 1 - \frac{\gamma}{2\beta} \right) \Delta t\, \ddot{u}_n \right]  = f_{n+1} 
\end{equation} 

which can be solved for $u_{n+1}$. 

#### Case B: Implicit Method without Damping $C = 0 $
If there is no dampening present in the governing equation one would expect the energy to be constant from one timestep to the next, this is however not always true due to artificial energy dissipation introduced by the time integration scheme. Numerical Dampening is however not necessarily a bad property, it can be used to dampend high-frequency oscillations and thus improve numerical stability and convergence at the cost of accuracy.


#### Case C: Explicit Method $C = 0, \beta = 0, \gamma = \frac{1}{2} $

Note that if $\beta = 0 $ and there is no damping ($C = 0 $), the scheme becomes explicit. In this case, we may take $\gamma = \frac{1}{2} $, and the update equations simplify to:

$
\mathbf{u}_{n+1} = \mathbf{u}_n + \dot{\mathbf{u}}_n \Delta t + \ddot{\mathbf{u}}_n \frac{\Delta t^2}{2}
$

$
\dot{\mathbf{u}}_{n+1} = \dot{\mathbf{u}}_n + (1 - \gamma) \ddot{\mathbf{u}}_n \Delta t + \gamma \ddot{\mathbf{u}}_{n+1} \Delta t
$

$
\ddot{\mathbf{u}}_{n+1} = \mathbf{M}^{-1} (\mathbf{f}_{n+1} - \mathbf{K} \mathbf{u}_{n+1})
$


### 3. Summary Table:
| Case | Method Type | Damping | Stability | Notes |
|------|-------------|---------|-----------|-------|
| A    | Implicit    | Yes     | Unconditional/Conditional ?| Realistic damping |
| B    | Implicit    | No      |Unconditional/Conditional ?| Energy-conserving |
| C    | Explicit    | No      | Unconditional/Conditional ? | Fast but limited |

## Task 2

#### Import Statements

In [3]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import assimulo.problem as apro
import matplotlib.pyplot as mpl
import assimulo.solvers as asol
from Second_Order import Second_Order
from Explicit_Problem_2nd import Explicit_Problem_2nd as EP2

In [4]:
# Define first order problem
k = 10
def elastic_pendulum(t,y):
    yvec = np.zeros_like(y)
    yvec[0] = y[2]
    yvec[1] = y[3]
    lam = k * (np.sqrt(y[0]**2 + y[1]**2) - 1) / np.sqrt(y[0]**2 + y[1]**2)
    yvec[2] = -1*y[0]*lam
    yvec[3] = -1*y[1]*lam - 1
    return yvec

initial_conditions = [0.0, 2.0, 0.0, 0.0]  # initial position (x,y) and velocity (vx, vy)
eP_Problem = apro.Explicit_Problem(elastic_pendulum, t0 = 0, y0 = initial_conditions)

EP2_Problem = EP2(eP_Problem, 2)


print("Acceleration from wrapper:", EP2_Problem.acceleration(0, initial_conditions[:2], initial_conditions[2:]))


Acceleration from wrapper: [ -0. -11.]


The Newmark method

In [7]:
# We already defined 2nd order problem as EP2_Problem, so we can directly call Newmark method on it. 
# We will implement Newmark method in the next step.

exp_sim = Second_Order(EP2_Problem, 0.0, 0.25)
exp_sim.reset()
simulation_time = 5
communication_points = 50
t, y = exp_sim.simulate(simulation_time, communication_points)
# Plot Results
plt.figure()

plt.plot(t, y[:,0], label=r'$x(t)$')
plt.plot(t, y[:,1], label=r'$y(t)$')
plt.plot(t, y[:,2], label=r'$v_x(t)$')
plt.plot(t, y[:,3], label=r'$v_y(t)$')

plt.xlabel('t')
plt.ylabel('state value')
plt.legend()
plt.title("BDF2: " + eP_Problem.name)
plt.grid(True)
plt.show()

shape y  (4,)
y:  [0. 2. 0. 0.]


AttributeError: 'Second_Order' object has no attribute 'dim'